In [2]:
import pandas as pd 
import numpy as np
import os 
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from scipy import stats


In [3]:
files = os.listdir('./Train_data')

In [4]:
train_sets = [pd.read_csv('./Train_data/'+i) for i in files]

In [5]:
cv_train_sets = [pd.read_csv('./Train_data/'+i) for i in files]

In [6]:
def trunc(x):
    return x.__trunc__()

In [7]:
for i in range(train_sets.__len__()):
    train_sets[i]['time_in_sec'] = (train_sets[i]['time']/1000).apply(trunc)
    train_sets[i]['group'] = i



In [8]:

for i in range(train_sets.__len__()):
    
    train_sets[i] = train_sets[i].groupby(['time_in_sec']).agg({
        'portPktOut':'mean',
        'portPktIn':'mean',
        'qSize':'mean',
        'label':lambda x:stats.mode(x)[0][0],
        'group':lambda x:i
        
    })

/tmp/ipykernel_11712/2594021348.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  'label':lambda x:stats.mode(x)[0][0],


In [9]:
train = pd.concat(train_sets)

In [10]:
agg_train = pd.DataFrame()
train.reset_index(inplace=True)

In [11]:
for column in train.columns:
    agg_train[column] = np.array(train[column])

In [12]:
train = agg_train

In [13]:
lgb = LGBMClassifier()

In [14]:
train

,time_in_sec,portPktOut,portPktIn,qSize,label,group
0,0,1191635.008,1191677.484,88737.195,6,0
1,1,994337.097,994294.621,27170.384,1,0
2,2,863208.613,863378.533,25891.828,1,0
3,3,1086619.436,1086459.876,119093.452,7,0
4,4,717895.174,717884.814,3998.960,8,0
...,...,...,...,...,...,...
9653,114,978051.462,978042.231,17663.919,7,77
9654,115,521298.647,521286.122,2526.794,4,77
9655,116,554594.940,554594.940,2383.836,9,77
9656,117,632640.960,632640.960,2880.080,9,77


In [15]:
# test data downsampling -------------------------------
test_file = os.listdir('./Test_data')
s = test_file.sort()
test_sets = [pd.read_csv('./Test_data/'+i) for i in test_file]


In [16]:

def func(x):
    return x.__trunc__()

In [17]:
for i in range(test_sets.__len__()):
    test_sets[i]['time_in_sec'] = (test_sets[i]['time']/1000).apply(func)
    test_sets[i]['time_str'] = [str(i) for i in np.array(test_sets[i]['time'].values)]
    test_sets[i]['group'] = test_file[i].lower().removesuffix('.csv') + '_' + test_sets[i]['time_str']



In [18]:
test_sets[0]

,time,portPktIn,portPktOut,qSize,time_in_sec,time_str,group
0,0,0.0,0.0,0,0,0,test0_0
1,1,371264.0,371264.0,0,0,1,test0_1
2,2,0.0,0.0,0,0,2,test0_2
3,3,265216.0,265216.0,0,0,3,test0_3
4,4,1167329.0,1167329.0,0,0,4,test0_4
...,...,...,...,...,...,...,...
128995,128995,967479.0,956083.0,11396,128,128995,test0_128995
128996,128996,1249434.0,1249434.0,11396,128,128996,test0_128996
128997,128997,1182864.0,1153987.0,40273,128,128997,test0_128997
128998,128998,1155450.0,1185363.0,10360,128,128998,test0_128998


In [19]:
test = pd.concat(test_sets)

In [20]:
id = test['group']

In [21]:
test.drop(columns=['time','group'],inplace=True)

In [22]:
gkf = GroupKFold()
flds = []
for train_indx, test_indx in   gkf.split(train.drop(columns=['label','group']),train['label'],train['group']):
    trainset = train.iloc[train_indx]
    val_set = train.iloc[test_indx]
    flds.append((trainset,val_set))



In [23]:
flds[0][0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7685 entries, 0 to 9538
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time_in_sec  7685 non-null   int64  
 1   portPktOut   7685 non-null   float64
 2   portPktIn    7685 non-null   float64
 3   qSize        7685 non-null   float64
 4   label        7685 non-null   int64  
 5   group        7685 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 420.3 KB


In [24]:


results = []
for i in range(flds.__len__()):
    lgb.fit(X=flds[i][0].drop(columns=['label','group']),y=flds[i][0]['label'])
    pred = lgb.predict(flds[i][1].drop(columns=['label','group']))
    results.append(accuracy_score(flds[i][1]['label'],pred))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 913
[LightGBM] [Info] Number of data points in the train set: 7685, number of used features: 4
[LightGBM] [Info] Start training from score -2.462390
[LightGBM] [Info] Start training from score -2.433796
[LightGBM] [Info] Start training from score -2.480881
[LightGBM] [Info] Start training from score -2.476226
[LightGBM] [Info] Start training from score -2.504485
[LightGBM] [Info] Start training from score -2.432313
[LightGBM] [Info] Start training from score -2.506079
[LightGBM] [Info] Start training from score -2.477775
[LightGBM] [Info] Start training from score -2.463918
[LightGBM] [Info] Start training from score -2.480881
[LightGBM] [Info] Start training from score -2.642577
[LightGBM] [Info] Start training from score -2.473135
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, th

In [25]:
np.mean(results)

0.3764619730876232

In [26]:
train

,time_in_sec,portPktOut,portPktIn,qSize,label,group
0,0,1191635.008,1191677.484,88737.195,6,0
1,1,994337.097,994294.621,27170.384,1,0
2,2,863208.613,863378.533,25891.828,1,0
3,3,1086619.436,1086459.876,119093.452,7,0
4,4,717895.174,717884.814,3998.960,8,0
...,...,...,...,...,...,...
9653,114,978051.462,978042.231,17663.919,7,77
9654,115,521298.647,521286.122,2526.794,4,77
9655,116,554594.940,554594.940,2383.836,9,77
9656,117,632640.960,632640.960,2880.080,9,77


In [27]:
train

,time_in_sec,portPktOut,portPktIn,qSize,label,group
0,0,1191635.008,1191677.484,88737.195,6,0
1,1,994337.097,994294.621,27170.384,1,0
2,2,863208.613,863378.533,25891.828,1,0
3,3,1086619.436,1086459.876,119093.452,7,0
4,4,717895.174,717884.814,3998.960,8,0
...,...,...,...,...,...,...
9653,114,978051.462,978042.231,17663.919,7,77
9654,115,521298.647,521286.122,2526.794,4,77
9655,116,554594.940,554594.940,2383.836,9,77
9656,117,632640.960,632640.960,2880.080,9,77


In [28]:
y = train['label']

In [29]:
train.drop(columns=['group','label'],inplace=True)
test.drop(columns=['time_str'],inplace=True)

In [30]:
test

,portPktIn,portPktOut,qSize,time_in_sec
0,0.0,0.0,0,0
1,371264.0,371264.0,0,0
2,0.0,0.0,0,0
3,265216.0,265216.0,0,0
4,1167329.0,1167329.0,0,0
...,...,...,...,...
118995,0.0,0.0,0,118
118996,0.0,0.0,0,118
118997,0.0,0.0,0,118
118998,0.0,0.0,0,118


In [31]:
lgb.fit(X=train,y=y)
prediction = lgb.predict(test)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 9658, number of used features: 4
[LightGBM] [Info] Start training from score -2.462586
[LightGBM] [Info] Start training from score -2.419773
[LightGBM] [Info] Start training from score -2.489681
[LightGBM] [Info] Start training from score -2.502244
[LightGBM] [Info] Start training from score -2.516248
[LightGBM] [Info] Start training from score -2.422104
[LightGBM] [Info] Start training from score -2.499719
[LightGBM] [Info] Start training from score -2.495943
[LightGBM] [Info] Start training from score -2.450508
[LightGBM] [Info] Start training from score -2.462586
[LightGBM] [Info] Start training from score -2.608869
[LightGBM] [Info] Start training from score -2.502244

In [ ]:
submition = pd.DataFrame({
    'ID':id,
    'Target':prediction
})

In [ ]:
submition.set_index('ID',inplace=True)

In [ ]:
submition.to_csv('submition.csv')